### Chapter 19

웹상에 있는 json 데이터를 불러와서 처리할 수 있다. 그런데 데이터를 좀 더 편리하게 접근할 수 있으면 좋겠다. 가령 `feed['Schedule']` 과 같은 구문은 불편하다. `feed.Schedule` 로 접근할 수 있도록 코드를 개선할 수 있다. 여기서는 FrozenJSON 클래스를 간단하게 구현하여 속성에 동적으로 접근할 수 있도록 한다.

In [ ]:
from collections import abc
from urllib.request import urlopen
import warnings
import os
import json
import reprlib

class FrozenJSON:
    #피드를 순환하면서 내포된 데이터들은 계속 FrozenJSON 으로 변환된다
    def __init__(self, mapping):
        self.__data=dict(mapping)
        #딕셔너리 생성해주기

    def __getattr__(self,name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        # 만약 그런 특성이 있으면 리턴해줌
        else:
            return FrozenJSON.build(self.__data[name])


    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj,abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

URL='http://www.oreilly.com/pub/sc/osconfeed'
JSON='data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg='downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)

    with urlopen(URL) as remote:
        local=open(JSON, 'wb')
        #JSON 경로에 파일 만들어 주면 작동함
        local.write(remote.read())

    with open(JSON, 'rb') as fp:
        #binary 파일이므로 binary로 읽어줌
        return json.load(fp)


raw_feed=load()
feed=FrozenJSON(raw_feed)
print(feed.Schedule.keys())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json


FileNotFoundError: ignored

그런데 `class` 와 같이, 파이썬에서 이미 예약어로 지정되어 불러올 수 없는 속성명일 경우가 있다. 그럴 때는 `iskeyword` 함수(keyword 모듈에 존재)를 사용하여, 파이썬의 키워드인지 확인하고 만약 파이썬의 키워드와 같은 이름의 함수명일 경우 `class_` 등으로 바꿔서 조회하게 해주는 코드를 짤 수 있다. 이렇게 하면 코드가 더 견고해진다. 또한 파이썬에서 정당한 식별자 이름이 아닌 경우의 속성도 생각해 볼 수 있는데, 이는 파이썬의 `isidentifier` 함수를 사용하여 식별할 수 있다.

+ `__new__` 메서드

흔히 `__init__` 메서드를 생성자 메서드라고 부르지만 실제로 객체를 생성하는 메서드는 `__new__` 이다. 이는 클래스 메서드로서 특별 취급되어 @classmethod 데커레이터를 사용하지 않아도 사용한 것처럼 동작한다. 그리고 `__new__` 메서드는 클래스를 인수로 받아서 그 클래스의 새로운 객체를 생성한다. 그 객체가 `__init__`의 첫 번째 인수 self로 전잘되는 것이다. 즉 실제로 생성자 역할을 하는 건 `__new__` 이며 `__init__`은 그저 초기화 함수 역할일 뿐이다.

+ 속성을 검증하는 프로퍼티

프로퍼티는 속성을 검증할 때 유용하게 쓰일 수 있다. 다른 언어의 getter와 setter 역할을 하는 것이다. 가령 농산물 상점의 품목명을 뜻하는 다음과 같은 클래스를 생각하자.




In [ ]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description=description
        self.weight=weight
        self.price=price

    def subtotal(self):
        return self.weight*self.price
    

아주 간단하다. 그러나 문제는 무게나 가격이 음수 input을 받을 경우 전혀 대응할 수 없다는 점이다. 만약 고객이 농산물 구매 사이트에서 구매할 무게를 음수로 입력한다면 문제가 생길 것이다. 이런 것을 방지할 수 있게 코드를 짜야 한다. 간단하게 weight에 프로퍼티를 사용하면 다음과 같은 코드가 된다.

In [ ]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description=description
        self.weight=weight
        self.price=price

    def subtotal(self):
        return self.weight*self.price

    @property
    def weight(self):
        return self.__weight

    @weight.setter #weight 속성의 세터
    def weight(self, value):
        if value>0:
            self.__weight=value
        else:
            raise ValueError('value must be larger than 0')

peanut=LineItem('peanut', -1, 10)
#수량이 -1로 입력되었으므로 세터에서 에러 발생
print(peanut)

이런 방식으로 수량이 음수가 될 수 있는 문제를 해결했다. 그런데 만약에 가격도 음수가 될 수 없도록 지정하고 싶다면 어떻게 할까? 물론 price 속성에 대해서도 프로퍼티를 만들어 주면 된다. 하지만 그런 속성이 많다면 프로퍼티 팩토리를 구현해 추상화시키는 것이 좋은 방향이다.

+ 프로퍼티 탐구

`@property` 는 데커레이터로 많이 사용되지만 사실상 클래스다. 참고로 함수와 클래스는 모두 콜러블이고 `new` 연산자가 파이썬에는 없으므로, 생성자를 호출하는 것은 팩토리 함수를 호출하는 것과 마찬가지고 따라서 클래스도 데커레이터로 사용할 수 있는 것이다.

이때 property 함수는 전체적으로 다음과 같다.

`property(fget=None, fset=None, fdel=None, doc=None)`

모든 인수는 선택적이다. 따라서 이 함수 구문을 이용해서
`weight=property(get_weight, set_weight)` 식으로 속성의 게터와 세터를 지정해 줄 수도 있다. `@`를 이용한 데커레이터가 파이썬 2.4에 생겼으므로 그전에는 이렇게 property 함수를 직접 사용하여 게터와 세터를 정의했다. 이러한 고전적인 구문이 프로퍼티 팩토리를 만들 때 쓰인다.

+ 객체 속성을 가리는 프로퍼티

프로퍼티는 클래스 객체의 속성에 대한 접근도 관리한다.
객체와 클래스가 동일한 이름의 속성을 가지고 있으면, 객체를 통해 속성에 접근할 때는 객체 속성이 클래스 속성을 가린다. 객체의 속성을 호출할 때, 객체의 클래스의 속성보다는 그 객체 자체의 속성이 우선 호출된다는 것이다. 예시 코드를 보자.



In [2]:
class Test1:
  data='it is a test text'

t1=Test1()
print(vars(t1))
print(t1.data)

# 하지만 객체에 같은 이름의 속성을 지정하면, 클래스 속성은 가려진다
t1.data='new data'
print(t1.data)
print(Test1.data)

{}
it is a test text
new data
it is a test text


하지만 프로퍼티로 지정한 속성은 객체 속성에 의해 가려지지 않는다.

In [4]:
class Test2:
  @property
  def data(self):
    return 'it is a test data'

t2=Test2()
print(Test2.data)
print(t2.data)

#단순히 t2.data 로 값을 설정하면 setter함수가 없으므로 에러가 발생한다. 따라서 __dict__에 직접 값을 넣어줘 본다.
t2.__dict__['data']='new data'
print(t2.data)
#새로운 값으로 가려지지 않는다

it is a test data
it is a test data


마찬가지로, property 함수를 직접 이용해서 새로운 프로퍼티를 지정하는 경우에도 그게 객체 속성을 가린다. 요점은 t2.attr 같은 표현식이 t2객체에서 시작해서 속성명을 검색하는 게 아니라는 것이다. 기본적으로는 객체의 클래스 안에 속성명의 프로퍼티가 없을 경우에만 객체 내에서 속성명을 검색한다. 클래스의 프로퍼티 속성들을 먼저 검색하고 나서 클래스 객체 내에서 속성을 검색하는 것이다.

+ 프로퍼티의 docs

콘솔의 help() 함수 등이 프로퍼티에 대한 문서를 보여주어야 할 때 프로퍼티는 `__doc__` 속성에서 정보를 가져온다. 고전적인 프로퍼티 호출 구문
`property(fget=None, fset=None, fdel=None, doc=None)`
에서는 doc 인수가 받은 문자열이 그 역할을 한다. 그리고 프로퍼티를 데커레이터로 사용하는 경우 @property 데커레이터로 장식된 게터의 문서화 문자열이 프로퍼티의 docs로 사용된다.

+ 프로퍼티 팩토리 만들기

이제 우리가 원래 의도했던, 다양한 조건의 속성들에 같은 게터/세터를 적용할 수 있는 프로퍼티 팩토리를 만들어 본다.

In [6]:
def quantity(storage_name):
  def qgetter(inst):
    return inst.__dict__[storage_name]
    #만약 inst.storage_name을 호출하게 되면 게터가 호출되어서 무한 재귀에 걸리게 되므로 __dict__에서 직접 호출해 준다.

  def qsetter(inst,value):
    if value>0:
      inst.__dict__[storage_name]=value
      #__dict__에서 호출해야 한다는 것을 꼭 기억하라
    else:
      raise ValueError('value must be larger than 0')

  return property(qgetter, qsetter)

class LineItem:
  weight=quantity('weight')
  price=quantity('price')
  #'weight' 와 'price'를 두 번 입력해야 하는 게 불편하지만 어쩔 수 없다. quantity가 변수에 어떤 속성을 바인딩해야 할지 알아야 하므로 그렇다. 
  #이걸 해결하기 위한 방법은 메타프로그래밍 기법을 사용해야 해서 복잡하다.

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price
  
  def subtotal(self):
    return self.weight*self.price



+ 속성을 처리하는 내장함수

`dir(odject)`
-> 객체 속성을 나열한다.

 `getattr(object, name, default)` ->object에서 name 문자열로 식별된 속성을 가져온다. 없으면 에러 혹은 default값 반환(있으면)

`hasattr(object, name)` -> 해당 이름의 속성이 있으면 True 반환. getattr를 호출하고 나서 예외가 발생하는지를 보는 식으로 구현되었다.

`setattr(object, name, value)` -> object가 허용하면 name 속성에 value 할당

`vars(object)` -> `object.__dict__` 를 반환한다. 이때 `__slots__`는 있고 `__dict__`는 없는 클래스의 객체는 처리 불가하다.

